In [1]:
#verificar com o ayres [[]] aninhado

import json
import re

data = []
with open('dump_small.jsonln', 'r') as file:
    for line in file:
        data.append(json.loads(line))
        
print(f'Numero de documentos: {len(data)}')

Numero de documentos: 11225


In [2]:
print(data[0].keys())

dict_keys(['title', 'body'])


In [3]:
print(data[0]['title'])

Alexandre (nome)


In [4]:
print(data[0]['body'])

{{Multitag|mnot|rev|data=outubro de 2019}}
{{Info/Nome
|imagem= Napoli BW 2013-05-16 16-24-01.jpg
|legenda= [[Alexandre, o Grande]], uma das pessoas notórias de nome '''Alexandre'''.
|gênero= Masculino
|significado= Protetor do Homem
|origem= Do [[Língua grega|grego]] Αλέξανδρος
|relativos= Alex; Alexander; Alessandro; Sandro
}}
'''Alexandre''' é um [[prenome]] popular da [[língua portuguesa]]. É [[cognato]] ao nome [[Alexander]], da [[língua inglesa]]. Em [[países lusófonos]], pessoas chamadas Alexandre são normalmente [[Alcunha|apelidadas]] de [[Alex]].

== Origem ==
O nome deriva do [[latim]] ''Alexander'', [[romanização]] do nome [[Língua grega|grego]] Αλέξανδρος (''Aléksandros''). [[Etimologia|Etimologicamente]], o nome seria um composto do [[verbo]] ''ἀλέξειν'' (''aléksein''), "defender", e o [[substantivo]] ''ἀνδρός'' (''andrós''), [[genitivo]] de ''ἀνήρ'' (''anēr''), "homem". Assim, pode ser traduzido como "protetor da Humanidade".{{carece de fontes|data=abril de 2017}}

O term

---

Como vocês podem ver, o texto está cheio de caracteres de estruturação de documento.

---

**Atividade:** Consulte a documentação da Wikipedia para descobrir o que são as marcações `[[`, `]]`, `{{` e ```}}```

**R:** O que está entre [[ ]] é um hypertexto. o que está entre {{}} é um supertexto (ex: aaaa², o 2 é um supertexto)

---

Vamos explorar o corpus para minerar itens de interesse.

**Exercicio:** Faça uma função que retorna uma lista com todos os links html de um texto deste corpus.

In [5]:
def find_links(text):
    return re.findall(r'\[\[.+?\]\]', text)
find_links(" evoluir.thumb|esquerda|300px|Primeira edi\u00e7\u00e3o do Unua Libro, em [[L\u00edngua russa|russo]]O primeiro livro sobre o esperanto foi")

['[[Língua russa|russo]]']

In [6]:
print(find_links(data[2]['body']))

['[[língua auxiliar]]', '[[língua artificial]]', '[[L. L. Zamenhof]]', '[[1887]]', '[[Línguas românicas]]', '[[línguas germânicas|germânicas]]', '[[línguas eslavas]]', '[[língua auxiliar|Linguagem auxiliar internacional]]', '[[Ethnologue]]', '[[Alfabeto latino]]', '[[Ortografia do esperanto|Alfabeto Esperanto]]', '[[Academia de Esperanto]]', '[[Imagem:Number_of_Esperanto_association_members_by_country.svg|300px]]', '[[língua planejada]]', '[[Esperantujo]]', '[[médico]]', '[[Judeus|judeu]]', '[[Ludwik Lejzer Zamenhof]]', '[[1887]]', '[[língua franca]]', '[[intercâmbio cultural]]', '[[Valor propedêutico do esperanto|evidências de que auxilia na aprendizagem dos demais idiomas]]', '[[língua auxiliar]]', '[[Radical (linguística)|radical]]', '[[latim]]', '[[desinência]]', '[[particípio presente]]', '[[substantivo]]', '[[Ludwik Lejzer Zamenhof]]', '[[Białystok]]', '[[Polônia]]', '[[Império Russo]]', '[[Imagem:Protoesperanto.png|miniaturadaimagem|Tabela comparando o protoEsperanto com o Esper

**Exercicio:** Faça uma função que recebe um texto do corpus e substitui todas as ocorrências de links da Wikipedia (textos entre os tags `[[` e `]]`) por texto simples, e retorna o documento limpo. Por exemplo, quando encontrar algo como `[[Etimologia|Etimologicamente]]` substituir por `Etimologicamente`. Você pode supor que os links não são *aninháveis*, como por exemplo `[[blablabla[[etc]]blebleble]]`.

In [7]:
def replace_links(text):
    
    clean = text
    
    links = find_links(text)
    words = []
    for i in links:
        a = re.search(r'(?<=\[\[).+?(?=\]\])', i)
        if '|' in a.group(0):
            b = a.group(0).rfind('|') + 1
        else:
            b = 0
        words.append(a.group(0)[b:])

    for i in range(len(links)):
        clean = clean.replace(links[i], words[i])
    
    return clean

**Exercicio**: Faça uma função que recebe um texto do corpus e remove todas as ocorrências de referências (textos entre os tags `<ref>` e `</ref>`).

In [8]:
def remove_ref(text):
    return re.sub(r'<ref.*>.*</ref>', '', text)

In [9]:
#replace vss sub
def remove_italic_bold(text):
    return re.sub(r"\'\'+", '', text)

In [10]:
def remove_endline(text):
    return text.replace('\n','')

In [11]:
def remove_double_equal(text):
    return text.replace('==','')

**Exercicio:** Faça uma função que recebe um texto do corpus e remove todos os templates (textos entre os tags `{{` e `}}`). Este exercício é desafiante: *os templates podem ser aninhados*! E agora, como proceder?

In [12]:
def remove_templates(text):
    a = text
    while re.search(r'\{\{[^\{\{\}\}]*\}\}', a):
        to_replace = re.findall(r'\{\{[^\{\{\}\}]*\}\}', a)
        for i in to_replace:
            a = a.replace(i, '')
    
    return a

**Exercicio:** Usando os codigos desenvolvidos acima, faça uma função que recebe um texto do corpus e retorna a sua versão limpa.

In [13]:
def clear_text(text):
    clean_links = replace_links(text)
    remove_quotes = remove_italic_bold(clean_links)
    remove_ref_ = remove_ref(remove_quotes)
    remove_endline_ = remove_endline(remove_ref_)
    remove_double_equal_ = remove_double_equal(remove_endline_)
    return remove_templates(remove_double_equal_)

**Exercicio:** Limpe todos os documentos e explore os resultados para ver o que mais dá para limpar. Nosso objetivo é ter uma coletânea de textos limpos para poder criar um vocabulário da língua portuguesa!

In [14]:
import json
with open('dump_small_clean.jsonln', 'w') as file:
    for i in range(10):
        json.dump({"body": clear_text(data[i]['body']), "title": data[i]["title"]}, file)

# Criando um tokenizador

Agora que temos um conjunto de documentos limpos, vamos analisar esses documentos para:

- Separar os documentos em palavras

- Separar os documentos em frases

**Exercicio:** Faça uma função que separa os documentos em palavras usando simplesmente os espaços em branco como indicador de separação.

**Exercicio:** Agora analise os resultados obtidos: essa separação em palavras está perfeita? Certamente não:

- Um problema óbvio é o seguinte: palavras ao final de uma sentença podem carregar consigo a pontuação da frase. 

- Mais ainda, algumas palavras no meio das sentenças podem ter vírgula e ponto-e-vírgula juntos.

- Sequências de emojis podem aparecer, e cada emoji deve ser considerado como uma palavra separada

Faça uma função que recebe um documento e faz uma separação melhor.